<a href="https://colab.research.google.com/github/SelgiAgilsa/Tugas3-Content-Based-Recomendation/blob/main/Tugas3_Content_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Filtering : Rekomendasi Studi Independen dan Magang Kampus Merdeka

In [73]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/SelgiAgilsa/Tugas3-Content-Based-Recomendation/main/data_rekomendasi.csv")
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran
0,Selgi Agilsa Ebimbi,"Jl. Nggepos III No.35A, Jerakah, Tugu, Semarang",Text Minning Machine Learning Jejaring Sosial ...
1,Braindevs PT. IlmuKomputerCom Braindevs Sistema,"Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto ...",Systems Analysis and Design Communication Skil...
2,Marawa Studio PT Mafela Ranca Wana,"Jl. Tan Malaka No. 138 Lt 3 , Kota Payakumbuh,...",Framing Shot Pemahaman Dasar Menggambar Perspe...
3,Suitmedia PT Suitmedia Kreasi Indonesia,"Jl. Pejaten Barat II No. 3A Jakarta, Indonesia...",Pemahaman terkait Kick off Meeting Project Tim...
4,Widya Robotics PT Widya Inovasi Indonesia,"Jl. Palagan Tentara Pelajar KM 7.5 No.31A, Mud...",Riset teknologi AI Dataset annotation Peranca...


## 1. Ikhtisar

In [74]:
df.describe()

,Nama Perusahaan,Alamat,Modul Pembelajaran
count,11,11,11
unique,11,10,11
top,Selgi Agilsa Ebimbi,Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tug...,Text Minning Machine Learning Jejaring Sosial ...
freq,1,2,1


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Nama Perusahaan     11 non-null     object
 1   Alamat              11 non-null     object
 2   Modul Pembelajaran  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


## 2. Modul Pembelajaran (Sebelum Preprocessing)

In [76]:
def print_description(index):
    example = df[df.index == index][['Modul Pembelajaran', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan :', example[1])
        print('Alamat:', example[2])

In [77]:
print_description(0)

Text Minning Machine Learning Jejaring Sosial Sistem Informasi Geografis Grafika Komputer Basis Data Terdistribusi Web Service
Nama Perusahaan : Selgi Agilsa Ebimbi
Alamat: Jl. Nggepos III No.35A, Jerakah, Tugu, Semarang


In [78]:
print_description(5)

Teamwork Collaboration Feature Store Analytical Thinking Tableau Access Management Communication 
Nama Perusahaan : tiket.com PT Global Tiket Network
Alamat: Wisma Barito Pacific II Lantai. 8, Jalan Letnan Jenderal Siswondo Parman Kavling. 60, Desa/Kelurahan Slipi, Kec. Pal Merah, Kota Adm. Jakarta Barat, Provinsi DKI Jakarta, Jakarta Raya Indonesia


In [79]:
print_description(10)

Soft Skill UI UX Design Pattern Pemrograman Mobile Code Versioning Dasar Dasar Pemrograman Databases Final Project
Nama Perusahaan : NF Computer PT Nurul Fikri Cipta Inovasi 
Alamat: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu,  Kec. Cimanggis, Kota Depok, Jawa Barat


## 3. Text Preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modulpembelajaran_clean'] = df['Modul Pembelajaran'].apply(clean_text)

In [81]:
df.head()

,Nama Perusahaan,Alamat,Modul Pembelajaran,modulpembelajaran_clean
0,Selgi Agilsa Ebimbi,"Jl. Nggepos III No.35A, Jerakah, Tugu, Semarang",Text Minning Machine Learning Jejaring Sosial ...,text minning machine learning jejaring sosial ...
1,Braindevs PT. IlmuKomputerCom Braindevs Sistema,"Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto ...",Systems Analysis and Design Communication Skil...,systems analysis and design communication skil...
2,Marawa Studio PT Mafela Ranca Wana,"Jl. Tan Malaka No. 138 Lt 3 , Kota Payakumbuh,...",Framing Shot Pemahaman Dasar Menggambar Perspe...,framing shot pemahaman dasar menggambar perspe...
3,Suitmedia PT Suitmedia Kreasi Indonesia,"Jl. Pejaten Barat II No. 3A Jakarta, Indonesia...",Pemahaman terkait Kick off Meeting Project Tim...,pemahaman terkait kick off meeting project tim...
4,Widya Robotics PT Widya Inovasi Indonesia,"Jl. Palagan Tentara Pelajar KM 7.5 No.31A, Mud...",Riset teknologi AI Dataset annotation Peranca...,riset teknologi ai dataset annotation peranca...


## 4. Modul Pembelajaran (Setelah Preprocessing)

In [82]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modulpembelajaran_clean', 'Nama Perusahaan','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama Perusahaan :', example[1])
        print('Alamat:', example[2])

In [83]:
print_description_clean(1)

systems analysis and design communication skills requirement analysis team work time management
Nama Perusahaan : Braindevs PT. IlmuKomputerCom Braindevs Sistema
Alamat: Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto Kav. 71-73 Pancoran Jakarta Selatan 12870


In [84]:
print_description_clean(3)

pemahaman terkait kick off meeting project timeline project management tools mengetahui cara interview dengan klien membuat minutes of meeting job request memahami scope of work pemahaman terkait maintenance services upselling nps pemahaman terkait company profile services pemahaman terkait tren industri digital digital assessment technical development pemahaman terkait quotation change request negosiasi dalam sales pemahaman terkait dokumen administrasi proyek weekly progress report handover report menemukan leads introductory email dan penggunaan tools hubspot crm
Nama Perusahaan : Suitmedia PT Suitmedia Kreasi Indonesia
Alamat: Jl. Pejaten Barat II No. 3A Jakarta, Indonesia 12510


In [85]:
print_description(10)

Soft Skill UI UX Design Pattern Pemrograman Mobile Code Versioning Dasar Dasar Pemrograman Databases Final Project
Nama Perusahaan : NF Computer PT Nurul Fikri Cipta Inovasi 
Alamat: Jl. Situ Indah No.116 RT. 006 RW. 010 Kel. Tugu,  Kec. Cimanggis, Kota Depok, Jawa Barat


## 5. TF-IDF & Cosine Similarity

In [86]:
df.set_index('Nama Perusahaan', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modulpembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.07277343, 0.        , 0.013862  , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.01387172, 0.        ,
        0.04638035, 0.05920045, 0.        , 0.        , 0.        ,
        0.02053276],
       [0.        , 0.        , 1.        , 0.04020069, 0.        ,
        0.        , 0.02298814, 0.00940582, 0.        , 0.01729139,
        0.02466622],
       [0.        , 0.01387172, 0.04020069, 1.        , 0.        ,
        0.00642717, 0.        , 0.01007845, 0.00831376, 0.02548158,
        0.01009195],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.1501657 , 0.        , 0.        ,
        0.        ],
       [0.        , 0.04638035, 0.        , 0.00642717, 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.07277343, 0.05920045, 0.02298814, 0.        , 0.

In [87]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:11]

0                                  Selgi Agilsa Ebimbi
1      Braindevs PT. IlmuKomputerCom Braindevs Sistema
2                   Marawa Studio PT Mafela Ranca Wana
3              Suitmedia PT Suitmedia Kreasi Indonesia
4            Widya Robotics PT Widya Inovasi Indonesia
5                    tiket.com PT Global Tiket Network
6     Metrodata Academy PT Mitra Integrasi Informatika
7              Power Academy PT Mitra Semeru Indonesia
8             NF Computer PT Nurul Fikri Cipta Inovasi
9           Hacktiv8 PT Hacktivate Teknologi Indonesia
10           NF Computer PT Nurul Fikri Cipta Inovasi 
Name: Nama Perusahaan, dtype: object

## 6. Modelling

In [88]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_kpsmerdeka = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_kpsmerdeka.append(list(df.index)[i])
        
    return recommended_kpsmerdeka

## 7. Prediksi

In [89]:
recommendations('Selgi Agilsa Ebimbi')

['Metrodata Academy PT Mitra Integrasi Informatika',
 'NF Computer PT Nurul Fikri Cipta Inovasi',
 'Braindevs PT. IlmuKomputerCom Braindevs Sistema']

In [90]:
recommendations("tiket.com PT Global Tiket Network")

['Braindevs PT. IlmuKomputerCom Braindevs Sistema',
 'Suitmedia PT Suitmedia Kreasi Indonesia',
 'Selgi Agilsa Ebimbi']

In [91]:
recommendations("Suitmedia PT Suitmedia Kreasi Indonesia")

['Marawa Studio PT Mafela Ranca Wana',
 'Hacktiv8 PT Hacktivate Teknologi Indonesia',
 'Braindevs PT. IlmuKomputerCom Braindevs Sistema']

# Selesai...